<a href="https://colab.research.google.com/github/AnnetVasileva/machineLearning/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [194]:
# import needed modules
import pandas as pd
import numpy as np

In [216]:
# we import the data from google drive, in case this is not the case for you please specify the directories yourself
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [196]:
# specifying directories for the files
train_set = '/content/drive/MyDrive/ML_ECO/cup98lrn.csv'  # training set
valid_set = '/content/drive/MyDrive/ML_ECO/cup98val.csv' # validation set
val_target = '/content/drive/MyDrive/ML_ECO/valtarget.csv' # labels for the validation set

In [197]:
# read in data
df = pd.read_csv(train_set,low_memory = False)  
val = pd.read_csv(valid_set,low_memory = False)  #ASSUMING THE ORDER of columns is the same
valtarget = pd.read_csv(val_target,low_memory = False)

In [198]:
# make CONTROLN to an index in both datasets and then merge val and valtarget

valtarget.index = valtarget.CONTROLN
valtarget = valtarget.drop('CONTROLN', axis = 1)
val.index = val.CONTROLN
val = val.drop('CONTROLN', axis = 1)

val = val.merge(valtarget, right_index= True, left_index=True)

In [199]:
# next step is making sure that the order of the columns in train and validation data sets is the same
df = df.drop('CONTROLN', axis = 1) # we do not need this anymore
val = val[df.columns]

In [200]:
# now we append the rows from validation dataset to the df
df = df.append(val)
df.reset_index(inplace= True, drop = True)

## Preprosessing
Our preprosessing will include 4 steps:
1. We will delete the columns we will not use (please see documentation to see which columns we drop)
2. Re-code /transform complex variables to easier ones
3. Encode ordinal variables using ordinal encoder and create one-hot-encoding where ordinal encoding is not suitable
4. Delete observations with missing data or additionally drop features that contain more than 40% of missing data

Please note: our dataset is highly imbalanced, TARGET_B, having values 1(donor) and 0 (not a donor) has a proportion of 5%/95%. This means that by justpredicting that everone is not a donor we already get an accuracy of 95%.



**STEP 1: Drop Features**

In dropping features we follow certain rules, namely:
1. We drop every feature that is a date
2. We drop features that refer to the sources of information 
3. We drop features that are already covered implicitly by the other features that we will encode 
4. We drop features that have a very complex structure and would require a lot of dummy variables (like STATE)
5. We drop features that highly depend on features that we previously decided to drop (like WEALTH2)
6. We drop variables too specific and try to keep those more general to learn an overall relationship.




In [201]:
# here we define some groups of variable
# for more info please consult the cup98dic.txt

yesno_variables = ['COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY',
       'STEREO', 'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS',
       'WALKER', 'KIDSTUFF', 'CARDS', 'PLATES']
rfas = ['RFA_2','RFA_3', 'RFA_4', 'RFA_5', 'RFA_6', 'RFA_7', 'RFA_8', 'RFA_9', 'RFA_10',
       'RFA_11', 'RFA_12', 'RFA_13', 'RFA_14', 'RFA_15', 'RFA_16', 'RFA_17',
       'RFA_18', 'RFA_19', 'RFA_20', 'RFA_21', 'RFA_22', 'RFA_23', 'RFA_24'] # RFA variables - decided to drop and keep onl the most recent one

drop = ['ODATEDW','OSOURCE','TCODE','STATE','ZIP','MAILCODE','PVASTATE','DOB','NOEXCH','RECINHSE','RECP3','RECPGVG','RECSWEEP','MDMAUD','WEALTH2','GEOCODE',
 'ADATE_2','ADATE_3','ADATE_4','ADATE_5','ADATE_6','ADATE_7','ADATE_8','ADATE_9','ADATE_10','ADATE_11','ADATE_12','ADATE_13',
 'ADATE_14','ADATE_15','ADATE_16','ADATE_17','ADATE_18','ADATE_19','ADATE_20','ADATE_21','ADATE_22','ADATE_23','ADATE_24',
 'RDATE_3','RDATE_4','RDATE_5','RDATE_6','RDATE_7', 'RDATE_8', 'RDATE_9','RDATE_10', 'RDATE_11', 'RDATE_12', 'RDATE_13', 'RDATE_14', 'RDATE_15',
 'RDATE_16','RDATE_17', 'RDATE_18', 'RDATE_19','RDATE_20', 'RDATE_21', 'RDATE_22', 'RDATE_23', 'RDATE_24', 'RAMNT_3', 'RAMNT_4', 'RAMNT_5', 'RAMNT_6',
 'RAMNT_7', 'RAMNT_8', 'RAMNT_9', 'RAMNT_10', 'RAMNT_11', 'RAMNT_12', 'RAMNT_13', 'RAMNT_14', 'RAMNT_15', 'RAMNT_17', 'RAMNT_18', 'RAMNT_19', 'RAMNT_20',
 'RAMNT_21', 'RAMNT_22', 'RAMNT_23', 'RAMNT_24', 'GEOCODE2', 'CLUSTER2', 'SOLP3', 'SOLIH', 'LASTDATE', 'FISTDATE', 'NEXTDATE', 'MALEMILI', 'MALEVET', 'VIETVETS',
 'WWIIVETS', 'LOCALGOV', 'STATEGOV', 'FEDGOV', 'MSA', 'ADI', 'DMA','CLUSTER', 'AGEFLAG', 'DATASRCE', 'LIFESRC', 'MINRDATE', 'MAXRDATE', 'MAXADATE', 'NUMCHLD',
 'ETH1', 'ETH2', 'ETH3','ETH4', 'ETH5', 'ETH6', 'ETH7', 'ETH8', 'ETH9', 'ETH10', 'ETH11',
  'ETH12', 'ETH13', 'ETH14', 'ETH15', 'ETH16', 
  'AGE902','AGE903', 'AGE904', 'AGE905', 'AGE906', 'AGE907', 'CHIL1', 'CHIL2',
  'CHIL3', 'AGEC1', 'AGEC2', 'AGEC3', 'AGEC4', 'AGEC5', 'AGEC6', 'AGEC7','CHILC1', 'CHILC2', 'CHILC3', 
  'CHILC5', 'HHAGE1', 'HHAGE2', 'HHAGE3', 'HHN1', 'HHN2', 'HHN3', 'HHN4','HHN5', 'HHN6', 'MARR1', 'MARR2', 'MARR3', 'MARR4', 
  'DW1', 'DW2', 'DW3', 'DW4', 'DW5', 'DW6', 'DW7', 'DW8', 'DW9',
  'HU2', 'HU3', 'HU4', 'HU5', 'HHD1', 'HHD2', 'HHD3', 'HHD4', 'HHD5',
  'HHD6', 'HHD7', 'HHD8', 'HHD9', 'HHD10', 'HHD11', 'HHD12', 
  'HUR1', 'HUR2', 'RHP1', 'RHP2', 'RHP3', 'RHP4', 'HUPA1', 'HUPA2',
  'HUPA3', 'HUPA4', 'HUPA5', 'HUPA6', 'HUPA7', 'RP1', 'RP2', 'RP3', 'RP4',
  'IC15', 'IC16', 'IC17', 'IC18', 'IC19', 'IC20', 'IC21', 'IC22', 'IC23', 
  'EIC1', 'EIC2', 'EIC3', 'EIC4', 'EIC5', 'EIC6', 'EIC7', 'EIC8', 'EIC9',
  'EIC10', 'EIC11', 'EIC12', 'EIC13', 'EIC14', 'EIC15', 'EIC16', 'OEDC1','OEDC2', 'OEDC3', 'OEDC4', 'OEDC5', 'OEDC6', 'OEDC7',
  'EC3', 'EC4', 'EC5', 'EC6', 'EC7', 'EC8', 'SEC1', 'SEC2', 'SEC3',
       'SEC4', 'SEC5', 'AFC1', 'AFC2', 'AFC3', 'AFC4', 'AFC5', 'AFC6', 'VC1',
       'VC2', 'VC3', 'VC4', 'ANC1', 'ANC2', 'ANC3', 'ANC4', 'ANC5', 'ANC6',
       'ANC7', 'ANC8', 'ANC9', 'ANC10', 'ANC11', 'ANC12', 'ANC13', 'ANC14','ANC15',
        'HC3', 'HC4', 'HC5', 'HC6', 'HC7', 'HC8', 'HC9', 'HC10', 'HC11', 'HC12',
       'HC13', 'HC14', 'HC15', 'HC16', 'HC17', 'HC18', 'HC19', 'HC20', 'HC21']

In [202]:
df = df.drop(drop, axis = 1)
df = df.drop(rfas, axis = 1)

**STEP 2 : Encode**

Now we start encoding things
1. We first rename the first column to Index
2. We then extract the first letter from DOMAIN to then encode it to one-hot encoding, the second letter will correspond to the Social Status (SES)
3. Afterwards we change the CHILDXX variables , which will be also encoded as one-hot 
4. We will then encode yes/no variables


In [203]:
df = df.rename(columns = {'Unnamed: 0' : 'Index'}) # rename first column to the Index

In [204]:
df.DOMAIN = df.DOMAIN.replace({' ': '99'}) # replace empty cells to 99 and then every 9 will be replaced to NaN

df['SES'] = 0 # NEW variable - socioeconomic status - second byte from DOMAIN
index = 0
for x in df.DOMAIN:
    df.SES[index] = int(df.DOMAIN[index][1])
    index += 1


# extracting the first byte
index = 0
for x in df.DOMAIN:
    df.DOMAIN[index] = df.DOMAIN[index][0]
    index += 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [205]:
df.HOMEOWNR = df.HOMEOWNR.replace({'H': 1, 'U': 0, ' ': np.NaN})
df['CHILD03'] = df['CHILD03'].replace({' ': 1, 'B': 2, 'F': 3, 'M':4}) # we will assume that no indication of 
# children means no children as a category
df['CHILD07'] = df['CHILD07'].replace({' ': 1, 'B': 2, 'F': 3, 'M':4})	
df['CHILD12'] = df['CHILD12'].replace({' ': 1, 'B': 2, 'F': 3, 'M':4})
df['CHILD18'] = df['CHILD18'].replace({' ': 1, 'B': 2, 'F': 3, 'M':4})#
df.PEPSTRFL = df.PEPSTRFL.replace({'X': 1, ' ':0})
df.DOMAIN = df.DOMAIN.replace({9 : np.NaN, '9': np.NaN})
df.DOMAIN = df.DOMAIN.replace({'S': 1, 'T':2, 'R':3, 'C':4, 'U':5})
df.SES = df.SES.replace({9 : np.NaN, 4: 3}) # please read documentation, the decision was made to replace every 4 to 3 so that technically 3 will include 
# all lowest SES
df.SES = df.SES.replace({3: 1, 1: 3}) # now we replace all 3 with 1 and all 1 with 3 so that we can decode it ordinally
df.GENDER = df.GENDER.replace({'A': 'U', 'C': 'U', ' ': np.NaN, 'J' : 'U'})
df.MDMAUD_F = df.MDMAUD_F.replace({'X': np.NaN})


for item in yesno_variables:
    df[item] = df[item].replace({' ': 0, 'N':0, 'Y': 1}) # these data values are a bit ambiguous since some of them do not have real N so we assume
    #that empty cells represent negative observations


**STEP 3: Delete values with nans**

We further get rid of the features that have more than 40% of missing values
We further drop the index since we wont need it


In [206]:
too_many_nas = [] # columns where we still have too many nans

for item in df:
    if df[item].isna().sum() > len(df)*0.4: # more tan 40% of all values is the threshold
      too_many_nas.append(str(item))

df = df.drop(too_many_nas, axis = 1)
df = df.drop('Index', axis = 1)

In [207]:
print(df.shape)

(191779, 156)


**STEP 4: One Hot Encoding and Categorical Encoding**

We first drop rows with nans and then specify features to encode.

In [208]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
#dropna
df = df.dropna()

one_hot_encoding = ['DOMAIN', 'CHILD03','CHILD12','CHILD18', 'GENDER', 'MDMAUD_R', 'RFA_2R', 'MAJOR'] 
categorical_encoding = ['MDMAUD_A', 'SES', 'INCOME','RFA_2F', 'RFA_2A']
categories = [[ 'X' ,'L', 'C', 'M', 'T'],
              ['1','2','3'],
              [1.,2.,3.,4.,5.,6.,7.],
              [1.0,2.0,3.0,4.0],
              ['A', 'B', 'C', 'D', 'E', 'F', 'G']]


ordinal_encoder = OrdinalEncoder(categories = categories)
one_hot_encoder = OneHotEncoder()

df_onehot = one_hot_encoder.fit_transform(df[one_hot_encoding])
df_ordinal = ordinal_encoder.fit_transform(df[categorical_encoding])


In [209]:
print(df.shape)

(111680, 156)


In [210]:
# TARGET VARIABLES
label_binary = df['TARGET_B'].to_numpy() # 1/0 donated or not
label_dollars = df['TARGET_D'].to_numpy() # how much donated


columns = np.setdiff1d(df.columns,one_hot_encoding)
columns = np.setdiff1d(columns,categorical_encoding)
columns = np.setdiff1d(columns,['TARGET_B','TARGET_D'])

df = df[columns]

In [211]:
print(df.shape)

(111680, 141)


In [212]:
df = np.concatenate((df,df_onehot.toarray(), df_ordinal), axis = 1)

In [213]:
print(df.shape)

(111680, 174)


In [217]:
# FINAL RESULT SHOULD BE df and a label
import pickle
data = {}
data['X'] = df
data['y'] = label_binary
pickle_path = '/content/drive/MyDrive/ML_ECO/data_set.pkl'
with open(pickle_path, "wb") as f:
    pickle.dump(data, f)